### CSC110 Final Project | University of Toronto

Source code distributed under the MIT License.    
Datasets distributed under their respective original licenses.

```
MIT License

Copyright (c) 2020 Mu "Samm" Du

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

```

# The Effect of Global Warming on Hurricane and Typhoon Occurrence

### Install & Import Dependencies

In [1]:
!pip install numpy
!pip install scikit-learn
!pip install plotly

You should consider upgrading via the '/home/samm/Documents/UofT/Year1/Courses/2020_Fall/CSC110/csc110_env/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/samm/Documents/UofT/Year1/Courses/2020_Fall/CSC110/csc110_env/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/samm/Documents/UofT/Year1/Courses/2020_Fall/CSC110/csc110_env/bin/python3.8 -m pip install --upgrade pip' command.


In [23]:
import csv
import pprint as pp
import math
import datetime as dt
import numpy as np
import sklearn as skl
import plotly.graph_objects as go
from plotly.subplots import make_subplots

### Preprocess Data

#### Preprocess Berkeley Earth Temperature Data

Note from the data file:
```
Temperatures are in Celsius and reported as anomalies 
relative to the Jan 1951-Dec 1980 average. Uncertainties represent the 95% confidence 
interval for statistical and spatial undersampling effects as well as ocean biases.
```

```
Estimated Jan 1951-Dec 1980 global mean temperature (C)
    Using air temperature above sea ice:   14.168 +/- 0.045
    Using water temperature below sea ice: 14.714 +/- 0.045
```

I will only use the dataset where sea ice temperatures are inferred from water temperatures, the specific choice is not relevant for my analysis.

In [3]:
# reference temperature
temp_ref = 14.714
temp_ref_unc = 0.045

# only retain relevant data
temp_data_trimmed = []
with open('data/berkeley-earth-global-temperature.txt', 'r') as temp_file:
    correct_line = False
    for line in temp_file:
        # only use data where Sea Ice Temperature Inferred from Water Temperatures
        if "Water Temperatures" in line:
            correct_line = True
        # skip header, empty lines, and data where Sea Ice Temperature Inferred from Air Temperatures,
        # and only use the year, month, temperature anomaly, and uncertainty values
        if correct_line and line != " \n" and line[0] != "%":
            temp_data_trimmed.append(line.strip().split()[:4])
        # print out omitted header
        elif correct_line:
            print("[HEADER]> ", line)

[HEADER]>  % Global Average Temperature Anomaly with Sea Ice Temperature Inferred from Water Temperatures

[HEADER]>  % 

[HEADER]>  %                  Monthly          Annual          Five-year        Ten-year        Twenty-year

[HEADER]>  % Year, Month,  Anomaly, Unc.,   Anomaly, Unc.,   Anomaly, Unc.,   Anomaly, Unc.,   Anomaly, Unc.

[HEADER]>   



The temperature data is organized as follows:
```
[year, month, temperature_anomaly, uncertainty]
```

In [4]:
# print out processed data for visual inspection
pp.pprint(temp_data_trimmed)

[['1850', '1', '-0.698', '0.385'],
 ['1850', '2', '-0.196', '0.372'],
 ['1850', '3', '-0.420', '0.355'],
 ['1850', '4', '-0.602', '0.311'],
 ['1850', '5', '-0.611', '0.218'],
 ['1850', '6', '-0.352', '0.240'],
 ['1850', '7', '-0.217', '0.248'],
 ['1850', '8', '-0.214', '0.244'],
 ['1850', '9', '-0.425', '0.210'],
 ['1850', '10', '-0.584', '0.223'],
 ['1850', '11', '-0.552', '0.237'],
 ['1850', '12', '-0.393', '0.289'],
 ['1851', '1', '-0.250', '0.329'],
 ['1851', '2', '-0.429', '0.320'],
 ['1851', '3', '-0.616', '0.259'],
 ['1851', '4', '-0.605', '0.245'],
 ['1851', '5', '-0.366', '0.355'],
 ['1851', '6', '-0.262', '0.236'],
 ['1851', '7', '-0.245', '0.202'],
 ['1851', '8', '-0.296', '0.214'],
 ['1851', '9', '-0.338', '0.243'],
 ['1851', '10', '-0.137', '0.275'],
 ['1851', '11', '-0.317', '0.233'],
 ['1851', '12', '-0.181', '0.339'],
 ['1852', '1', '-0.285', '0.420'],
 ['1852', '2', '-0.530', '0.314'],
 ['1852', '3', '-0.737', '0.361'],
 ['1852', '4', '-0.529', '0.278'],
 ['1852', '5',

In [5]:
print(len(temp_data_trimmed))

2050


I will apply the reference temperature to the monthly anomaly data, and compute the combined uncertainties.

In [6]:
temp_data = []
for line in temp_data_trimmed:
    
    line_proc = []
    
    # convert the year value to an integer
    line_proc.append(int(line[0]))
    
    # convert the month value to an integer
    line_proc.append(int(line[1]))
    
    # apply the reference temperature to the monthly anomaly
    line_proc.append(float(line[2]) + temp_ref)
    
    # compute the combined uncertainties between the monthly uncertainty and the reference uncertainty
    line_proc.append(
        math.sqrt(
            float(line[3]) ** 2 + temp_ref_unc ** 2
        )
    )
    
    temp_data.append(line_proc)
    

The processed temperature data is now organized as follows:
```
[year, month, monthly_temerature_celsius, uncertainty]
```

In [7]:
# print out processed data for visual inspection
pp.pprint(temp_data)

[[1850, 1, 14.016, 0.3876209488662861],
 [1850, 2, 14.518, 0.3747118893229837],
 [1850, 3, 14.294, 0.3578407467016578],
 [1850, 4, 14.112, 0.31423876272668844],
 [1850, 5, 14.103, 0.22259604668547012],
 [1850, 6, 14.362, 0.2441823089414956],
 [1850, 7, 14.497, 0.2520495982936692],
 [1850, 8, 14.5, 0.24811489274124598],
 [1850, 9, 14.289, 0.21476731594914528],
 [1850, 10, 14.13, 0.22749505489130967],
 [1850, 11, 14.162, 0.2412343259156955],
 [1850, 12, 14.321, 0.2924824781076637],
 [1851, 1, 14.464, 0.33206324698767853],
 [1851, 2, 14.285, 0.3231485726411305],
 [1851, 3, 14.098, 0.26288020085202307],
 [1851, 4, 14.109, 0.249098374141623],
 [1851, 5, 14.348, 0.3578407467016578],
 [1851, 6, 14.452, 0.2402519510846894],
 [1851, 7, 14.469000000000001, 0.20695168518279816],
 [1851, 8, 14.418000000000001, 0.21868013169924697],
 [1851, 9, 14.376000000000001, 0.24713154391942765],
 [1851, 10, 14.577, 0.2786574958618555],
 [1851, 11, 14.397, 0.23730571000294115],
 [1851, 12, 14.533000000000001, 

Plotting the Berkeley Earth Global Temperature Data

In [33]:
fig = make_subplots(rows=1, cols=1)

fig.add_trace(
    go.Scatter(
        x=[
            dt.datetime(year=line[0], month=line[1], day=1)
            for line in temp_data
        ],
        y=[line[2] for line in temp_data],
        mode="lines",
        line=go.scatter.Line(),
        name="Global Temperature"
    )
)

fig.add_trace(
    go.Scatter(
        x=[dt.datetime(year=1850, month=1, day=1), dt.datetime(year=2020, month=10, day=1)],
        y=[temp_ref, temp_ref],
        mode="lines",
        line=go.scatter.Line(color="gray"),
        name="Jan 1951-Dec 1980 <br>global mean temp."
    )
)

fig.update_layout(
    title="Global Temperature Data 1850-2014",
    xaxis_title="Year and Month",
    yaxis_title="Temperature (C)",
)

fig.show()

#### Preprocess NOAA Hurricane and Typhoon Data

In [8]:
def read_noaa_data(filename):
    """Read the NOAA Hurricane and Typhoon data given by the filename."""
    
    noaa_data = []
    
    with open(filename, 'r') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        
        for line in csv_reader:
            
            # print out omitted header
            if line_count == 0:
                print("[HEADER]> ", ", ".join(line))
                line_count += 1
            
            # only use the ID and the date values
            else:
                noaa_data.append([line[0], line[2]])
    
    return noaa_data

In [9]:
# read and process NOAA data
ht_data_atlantic = read_noaa_data('data/noaa-atlantic.csv')
ht_data_pacific = read_noaa_data('data/noaa-pacific.csv')

[HEADER]>  ID, Name, Date, Time, Event, Status, Latitude, Longitude, Maximum Wind, Minimum Pressure, Low Wind NE, Low Wind SE, Low Wind SW, Low Wind NW, Moderate Wind NE, Moderate Wind SE, Moderate Wind SW, Moderate Wind NW, High Wind NE, High Wind SE, High Wind SW, High Wind NW
[HEADER]>  ID, Name, Date, Time, Event, Status, Latitude, Longitude, Maximum Wind, Minimum Pressure, Low Wind NE, Low Wind SE, Low Wind SW, Low Wind NW, Moderate Wind NE, Moderate Wind SE, Moderate Wind SW, Moderate Wind NW, High Wind NE, High Wind SE, High Wind SW, High Wind NW


In [10]:
# print out processed atlantic data for visual inspection
pp.pprint(ht_data_atlantic)

[['AL011851', '18510625'],
 ['AL011851', '18510625'],
 ['AL011851', '18510625'],
 ['AL011851', '18510625'],
 ['AL011851', '18510625'],
 ['AL011851', '18510626'],
 ['AL011851', '18510626'],
 ['AL011851', '18510626'],
 ['AL011851', '18510626'],
 ['AL011851', '18510627'],
 ['AL011851', '18510627'],
 ['AL011851', '18510627'],
 ['AL011851', '18510627'],
 ['AL011851', '18510628'],
 ['AL021851', '18510705'],
 ['AL031851', '18510710'],
 ['AL041851', '18510816'],
 ['AL041851', '18510816'],
 ['AL041851', '18510816'],
 ['AL041851', '18510816'],
 ['AL041851', '18510817'],
 ['AL041851', '18510817'],
 ['AL041851', '18510817'],
 ['AL041851', '18510817'],
 ['AL041851', '18510818'],
 ['AL041851', '18510818'],
 ['AL041851', '18510818'],
 ['AL041851', '18510818'],
 ['AL041851', '18510819'],
 ['AL041851', '18510819'],
 ['AL041851', '18510819'],
 ['AL041851', '18510819'],
 ['AL041851', '18510820'],
 ['AL041851', '18510820'],
 ['AL041851', '18510820'],
 ['AL041851', '18510820'],
 ['AL041851', '18510821'],
 

 ['AL031863', '18630823'],
 ['AL031863', '18630823'],
 ['AL041863', '18630827'],
 ['AL041863', '18630827'],
 ['AL041863', '18630827'],
 ['AL041863', '18630827'],
 ['AL041863', '18630828'],
 ['AL041863', '18630828'],
 ['AL041863', '18630828'],
 ['AL041863', '18630828'],
 ['AL051863', '18630909'],
 ['AL051863', '18630909'],
 ['AL051863', '18630909'],
 ['AL051863', '18630909'],
 ['AL051863', '18630910'],
 ['AL051863', '18630910'],
 ['AL051863', '18630910'],
 ['AL051863', '18630910'],
 ['AL051863', '18630911'],
 ['AL051863', '18630911'],
 ['AL051863', '18630911'],
 ['AL051863', '18630911'],
 ['AL051863', '18630912'],
 ['AL051863', '18630912'],
 ['AL051863', '18630912'],
 ['AL051863', '18630912'],
 ['AL051863', '18630913'],
 ['AL051863', '18630913'],
 ['AL051863', '18630913'],
 ['AL051863', '18630913'],
 ['AL051863', '18630914'],
 ['AL051863', '18630914'],
 ['AL051863', '18630914'],
 ['AL051863', '18630914'],
 ['AL051863', '18630915'],
 ['AL051863', '18630915'],
 ['AL051863', '18630915'],
 

 ['AL081878', '18781011'],
 ['AL081878', '18781011'],
 ['AL081878', '18781011'],
 ['AL081878', '18781012'],
 ['AL081878', '18781012'],
 ['AL081878', '18781012'],
 ['AL081878', '18781012'],
 ['AL081878', '18781013'],
 ['AL081878', '18781013'],
 ['AL081878', '18781013'],
 ['AL081878', '18781013'],
 ['AL081878', '18781014'],
 ['AL081878', '18781014'],
 ['AL081878', '18781014'],
 ['AL081878', '18781014'],
 ['AL081878', '18781015'],
 ['AL081878', '18781015'],
 ['AL081878', '18781015'],
 ['AL081878', '18781015'],
 ['AL091878', '18781009'],
 ['AL091878', '18781009'],
 ['AL091878', '18781009'],
 ['AL091878', '18781009'],
 ['AL091878', '18781010'],
 ['AL091878', '18781010'],
 ['AL091878', '18781010'],
 ['AL091878', '18781010'],
 ['AL091878', '18781011'],
 ['AL091878', '18781011'],
 ['AL091878', '18781011'],
 ['AL091878', '18781011'],
 ['AL091878', '18781012'],
 ['AL091878', '18781012'],
 ['AL091878', '18781012'],
 ['AL091878', '18781012'],
 ['AL091878', '18781013'],
 ['AL091878', '18781013'],
 

 ['AL041884', '18841008'],
 ['AL041884', '18841008'],
 ['AL041884', '18841008'],
 ['AL041884', '18841009'],
 ['AL041884', '18841009'],
 ['AL041884', '18841009'],
 ['AL041884', '18841009'],
 ['AL041884', '18841010'],
 ['AL041884', '18841010'],
 ['AL041884', '18841010'],
 ['AL041884', '18841010'],
 ['AL041884', '18841011'],
 ['AL041884', '18841011'],
 ['AL041884', '18841011'],
 ['AL041884', '18841011'],
 ['AL041884', '18841012'],
 ['AL041884', '18841012'],
 ['AL041884', '18841012'],
 ['AL041884', '18841012'],
 ['AL041884', '18841013'],
 ['AL041884', '18841013'],
 ['AL041884', '18841013'],
 ['AL041884', '18841013'],
 ['AL041884', '18841014'],
 ['AL041884', '18841014'],
 ['AL041884', '18841014'],
 ['AL041884', '18841014'],
 ['AL041884', '18841015'],
 ['AL041884', '18841015'],
 ['AL041884', '18841015'],
 ['AL041884', '18841015'],
 ['AL041884', '18841016'],
 ['AL041884', '18841016'],
 ['AL041884', '18841016'],
 ['AL041884', '18841016'],
 ['AL041884', '18841017'],
 ['AL041884', '18841017'],
 

 ['AL051892', '18920916'],
 ['AL051892', '18920916'],
 ['AL051892', '18920916'],
 ['AL051892', '18920916'],
 ['AL051892', '18920917'],
 ['AL051892', '18920917'],
 ['AL051892', '18920917'],
 ['AL051892', '18920917'],
 ['AL051892', '18920918'],
 ['AL051892', '18920918'],
 ['AL051892', '18920918'],
 ['AL051892', '18920918'],
 ['AL051892', '18920919'],
 ['AL051892', '18920919'],
 ['AL051892', '18920919'],
 ['AL051892', '18920919'],
 ['AL051892', '18920920'],
 ['AL051892', '18920920'],
 ['AL051892', '18920920'],
 ['AL051892', '18920920'],
 ['AL051892', '18920921'],
 ['AL051892', '18920921'],
 ['AL051892', '18920921'],
 ['AL051892', '18920921'],
 ['AL051892', '18920922'],
 ['AL051892', '18920922'],
 ['AL051892', '18920922'],
 ['AL051892', '18920922'],
 ['AL051892', '18920923'],
 ['AL051892', '18920923'],
 ['AL051892', '18920923'],
 ['AL051892', '18920923'],
 ['AL061892', '18920925'],
 ['AL061892', '18920925'],
 ['AL061892', '18920925'],
 ['AL061892', '18920926'],
 ['AL061892', '18920926'],
 

...

 ['AL051999', '19990901'],
 ['AL051999', '19990901'],
 ['AL051999', '19990902'],
 ['AL051999', '19990902'],
 ['AL051999', '19990902'],
 ['AL051999', '19990902'],
 ['AL051999', '19990903'],
 ['AL051999', '19990903'],
 ['AL051999', '19990903'],
 ['AL051999', '19990903'],
 ['AL051999', '19990904'],
 ['AL051999', '19990904'],
 ['AL051999', '19990904'],
 ['AL051999', '19990904'],
 ['AL051999', '19990904'],
 ['AL051999', '19990905'],
 ['AL051999', '19990905'],
 ['AL051999', '19990905'],
 ['AL051999', '19990905'],
 ['AL051999', '19990906'],
 ['AL051999', '19990906'],
 ['AL051999', '19990906'],
 ['AL051999', '19990906'],
 ['AL051999', '19990907'],
 ['AL051999', '19990907'],
 ['AL051999', '19990907'],
 ['AL051999', '19990907'],
 ['AL051999', '19990908'],
 ['AL051999', '19990908'],
 ['AL051999', '19990908'],
 ['AL051999', '19990908'],
 ['AL061999', '19990824'],
 ['AL061999', '19990824'],
 ['AL061999', '19990824'],
 ['AL061999', '19990825'],
 ['AL061999', '19990825'],
 ['AL061999', '19990825'],
 

 ['AL162003', '20031006'],
 ['AL162003', '20031006'],
 ['AL162003', '20031007'],
 ['AL162003', '20031007'],
 ['AL162003', '20031007'],
 ['AL162003', '20031007'],
 ['AL162003', '20031008'],
 ['AL162003', '20031008'],
 ['AL162003', '20031008'],
 ['AL162003', '20031008'],
 ['AL162003', '20031009'],
 ['AL162003', '20031009'],
 ['AL162003', '20031009'],
 ['AL162003', '20031009'],
 ['AL162003', '20031010'],
 ['AL172003', '20030927'],
 ['AL172003', '20030928'],
 ['AL172003', '20030928'],
 ['AL172003', '20030928'],
 ['AL172003', '20030928'],
 ['AL172003', '20030929'],
 ['AL172003', '20030929'],
 ['AL172003', '20030929'],
 ['AL172003', '20030929'],
 ['AL172003', '20030930'],
 ['AL172003', '20030930'],
 ['AL172003', '20030930'],
 ['AL172003', '20030930'],
 ['AL172003', '20031001'],
 ['AL172003', '20031001'],
 ['AL172003', '20031001'],
 ['AL172003', '20031001'],
 ['AL172003', '20031002'],
 ['AL172003', '20031002'],
 ['AL172003', '20031002'],
 ['AL172003', '20031002'],
 ['AL172003', '20031003'],
 

 ['AL102005', '20050815'],
 ['AL102005', '20050815'],
 ['AL102005', '20050815'],
 ['AL102005', '20050815'],
 ['AL102005', '20050816'],
 ['AL102005', '20050816'],
 ['AL102005', '20050816'],
 ['AL102005', '20050816'],
 ['AL102005', '20050817'],
 ['AL102005', '20050817'],
 ['AL102005', '20050817'],
 ['AL102005', '20050817'],
 ['AL102005', '20050818'],
 ['AL102005', '20050818'],
 ['AL102005', '20050818'],
 ['AL112005', '20050822'],
 ['AL112005', '20050822'],
 ['AL112005', '20050823'],
 ['AL112005', '20050823'],
 ['AL112005', '20050823'],
 ['AL112005', '20050823'],
 ['AL122005', '20050823'],
 ['AL122005', '20050824'],
 ['AL122005', '20050824'],
 ['AL122005', '20050824'],
 ['AL122005', '20050824'],
 ['AL122005', '20050825'],
 ['AL122005', '20050825'],
 ['AL122005', '20050825'],
 ['AL122005', '20050825'],
 ['AL122005', '20050825'],
 ['AL122005', '20050826'],
 ['AL122005', '20050826'],
 ['AL122005', '20050826'],
 ['AL122005', '20050826'],
 ['AL122005', '20050827'],
 ['AL122005', '20050827'],
 

 ['AL082009', '20090926'],
 ['AL092009', '20090927'],
 ['AL092009', '20090928'],
 ['AL092009', '20090928'],
 ['AL092009', '20090928'],
 ['AL092009', '20090928'],
 ['AL092009', '20090929'],
 ['AL092009', '20090929'],
 ['AL092009', '20090929'],
 ['AL092009', '20090929'],
 ['AL092009', '20090930'],
 ['AL092009', '20090930'],
 ['AL092009', '20090930'],
 ['AL092009', '20090930'],
 ['AL092009', '20091001'],
 ['AL092009', '20091001'],
 ['AL092009', '20091001'],
 ['AL092009', '20091001'],
 ['AL092009', '20091002'],
 ['AL092009', '20091002'],
 ['AL092009', '20091002'],
 ['AL092009', '20091002'],
 ['AL092009', '20091003'],
 ['AL092009', '20091003'],
 ['AL092009', '20091003'],
 ['AL092009', '20091003'],
 ['AL092009', '20091004'],
 ['AL092009', '20091004'],
 ['AL092009', '20091004'],
 ['AL092009', '20091004'],
 ['AL092009', '20091005'],
 ['AL092009', '20091005'],
 ['AL092009', '20091005'],
 ['AL092009', '20091005'],
 ['AL092009', '20091006'],
 ['AL092009', '20091006'],
 ['AL092009', '20091006'],
 

 ['AL142011', '20110909'],
 ['AL142011', '20110909'],
 ['AL142011', '20110909'],
 ['AL142011', '20110909'],
 ['AL142011', '20110910'],
 ['AL142011', '20110910'],
 ['AL142011', '20110910'],
 ['AL142011', '20110910'],
 ['AL142011', '20110911'],
 ['AL142011', '20110911'],
 ['AL142011', '20110911'],
 ['AL142011', '20110911'],
 ['AL142011', '20110912'],
 ['AL142011', '20110912'],
 ['AL142011', '20110912'],
 ['AL142011', '20110912'],
 ['AL142011', '20110913'],
 ['AL142011', '20110913'],
 ['AL142011', '20110913'],
 ['AL142011', '20110913'],
 ['AL142011', '20110914'],
 ['AL142011', '20110914'],
 ['AL142011', '20110914'],
 ['AL142011', '20110914'],
 ['AL142011', '20110915'],
 ['AL142011', '20110915'],
 ['AL142011', '20110915'],
 ['AL142011', '20110915'],
 ['AL142011', '20110916'],
 ['AL142011', '20110916'],
 ['AL142011', '20110916'],
 ['AL142011', '20110916'],
 ['AL142011', '20110916'],
 ['AL152011', '20110906'],
 ['AL152011', '20110907'],
 ['AL152011', '20110907'],
 ['AL152011', '20110907'],
 

In [11]:
# print out processed pacific data for visual inspection
pp.pprint(ht_data_pacific)

[['EP011949', '19490611'],
 ['EP011949', '19490611'],
 ['EP011949', '19490611'],
 ['EP011949', '19490611'],
 ['EP011949', '19490612'],
 ['EP011949', '19490612'],
 ['EP011949', '19490612'],
 ['EP021949', '19490617'],
 ['EP021949', '19490617'],
 ['EP021949', '19490618'],
 ['EP021949', '19490618'],
 ['EP021949', '19490618'],
 ['EP021949', '19490618'],
 ['EP021949', '19490619'],
 ['EP021949', '19490619'],
 ['EP021949', '19490619'],
 ['EP021949', '19490619'],
 ['EP021949', '19490620'],
 ['EP021949', '19490620'],
 ['EP021949', '19490620'],
 ['EP021949', '19490620'],
 ['EP021949', '19490621'],
 ['EP021949', '19490621'],
 ['EP021949', '19490621'],
 ['EP021949', '19490621'],
 ['EP021949', '19490622'],
 ['EP021949', '19490622'],
 ['EP021949', '19490622'],
 ['EP021949', '19490622'],
 ['EP021949', '19490623'],
 ['EP021949', '19490623'],
 ['EP021949', '19490623'],
 ['EP031949', '19490903'],
 ['EP031949', '19490903'],
 ['EP031949', '19490904'],
 ['EP031949', '19490904'],
 ['EP031949', '19490904'],
 

 ['EP041954', '19540725'],
 ['EP041954', '19540725'],
 ['EP041954', '19540725'],
 ['EP041954', '19540726'],
 ['EP041954', '19540726'],
 ['EP041954', '19540726'],
 ['EP041954', '19540726'],
 ['EP041954', '19540727'],
 ['EP041954', '19540727'],
 ['EP041954', '19540727'],
 ['EP041954', '19540727'],
 ['EP041954', '19540728'],
 ['EP041954', '19540728'],
 ['EP041954', '19540728'],
 ['EP041954', '19540728'],
 ['EP041954', '19540729'],
 ['EP041954', '19540729'],
 ['EP041954', '19540729'],
 ['EP041954', '19540729'],
 ['EP041954', '19540730'],
 ['EP041954', '19540730'],
 ['EP041954', '19540730'],
 ['EP041954', '19540730'],
 ['EP041954', '19540731'],
 ['EP041954', '19540731'],
 ['EP041954', '19540731'],
 ['EP041954', '19540731'],
 ['EP041954', '19540801'],
 ['EP041954', '19540801'],
 ['EP041954', '19540801'],
 ['EP051954', '19540902'],
 ['EP051954', '19540902'],
 ['EP051954', '19540902'],
 ['EP051954', '19540902'],
 ['EP051954', '19540903'],
 ['EP051954', '19540903'],
 ['EP051954', '19540903'],
 

 ['EP161967', '19671031'],
 ['EP161967', '19671031'],
 ['EP161967', '19671101'],
 ['EP161967', '19671101'],
 ['EP161967', '19671101'],
 ['EP161967', '19671101'],
 ['EP161967', '19671102'],
 ['EP161967', '19671102'],
 ['EP161967', '19671102'],
 ['EP161967', '19671102'],
 ['EP161967', '19671103'],
 ['EP161967', '19671103'],
 ['EP161967', '19671103'],
 ['EP011968', '19680620'],
 ['EP011968', '19680620'],
 ['EP011968', '19680621'],
 ['EP011968', '19680621'],
 ['EP011968', '19680621'],
 ['EP011968', '19680621'],
 ['EP011968', '19680622'],
 ['EP021968', '19680704'],
 ['EP021968', '19680704'],
 ['EP021968', '19680704'],
 ['EP021968', '19680704'],
 ['EP021968', '19680705'],
 ['EP021968', '19680705'],
 ['EP021968', '19680705'],
 ['EP021968', '19680705'],
 ['EP021968', '19680706'],
 ['EP021968', '19680706'],
 ['EP021968', '19680706'],
 ['EP021968', '19680706'],
 ['EP021968', '19680707'],
 ['EP021968', '19680707'],
 ['EP021968', '19680707'],
 ['EP021968', '19680707'],
 ['EP021968', '19680708'],
 

 ['EP121968', '19680831'],
 ['EP121968', '19680901'],
 ['EP121968', '19680901'],
 ['EP121968', '19680901'],
 ['EP121968', '19680901'],
 ['EP121968', '19680902'],
 ['EP121968', '19680902'],
 ['EP121968', '19680902'],
 ['EP121968', '19680902'],
 ['EP121968', '19680903'],
 ['EP121968', '19680903'],
 ['EP121968', '19680903'],
 ['EP121968', '19680903'],
 ['EP121968', '19680904'],
 ['EP121968', '19680904'],
 ['EP121968', '19680904'],
 ['EP121968', '19680904'],
 ['EP121968', '19680905'],
 ['EP121968', '19680905'],
 ['EP121968', '19680905'],
 ['EP121968', '19680905'],
 ['EP121968', '19680906'],
 ['EP131968', '19680829'],
 ['EP131968', '19680829'],
 ['EP131968', '19680829'],
 ['EP131968', '19680829'],
 ['EP131968', '19680830'],
 ['EP131968', '19680830'],
 ['EP131968', '19680830'],
 ['EP141968', '19680909'],
 ['EP141968', '19680909'],
 ['EP141968', '19680909'],
 ['EP141968', '19680909'],
 ['EP141968', '19680910'],
 ['EP141968', '19680910'],
 ['EP141968', '19680910'],
 ['EP141968', '19680910'],
 

 ['EP071976', '19760809'],
 ['EP071976', '19760809'],
 ['EP071976', '19760809'],
 ['EP071976', '19760809'],
 ['EP071976', '19760810'],
 ['EP071976', '19760810'],
 ['EP071976', '19760810'],
 ['EP071976', '19760810'],
 ['EP071976', '19760811'],
 ['EP071976', '19760811'],
 ['EP071976', '19760811'],
 ['EP071976', '19760811'],
 ['EP071976', '19760812'],
 ['EP071976', '19760812'],
 ['EP071976', '19760812'],
 ['EP071976', '19760812'],
 ['EP071976', '19760813'],
 ['EP071976', '19760813'],
 ['EP071976', '19760813'],
 ['EP071976', '19760813'],
 ['EP071976', '19760814'],
 ['EP071976', '19760814'],
 ['EP071976', '19760814'],
 ['EP071976', '19760814'],
 ['EP071976', '19760815'],
 ['EP071976', '19760815'],
 ['EP071976', '19760815'],
 ['EP071976', '19760815'],
 ['EP071976', '19760816'],
 ['EP071976', '19760816'],
 ['EP071976', '19760816'],
 ['EP071976', '19760816'],
 ['EP071976', '19760817'],
 ['EP071976', '19760817'],
 ['EP071976', '19760817'],
 ['EP071976', '19760817'],
 ['EP071976', '19760818'],
 

...

 ['EP051998', '19980728'],
 ['EP051998', '19980729'],
 ['EP051998', '19980729'],
 ['EP051998', '19980729'],
 ['EP051998', '19980729'],
 ['EP051998', '19980730'],
 ['EP051998', '19980730'],
 ['EP051998', '19980730'],
 ['EP051998', '19980730'],
 ['EP051998', '19980731'],
 ['EP051998', '19980731'],
 ['EP051998', '19980731'],
 ['EP051998', '19980731'],
 ['EP051998', '19980801'],
 ['EP061998', '19980729'],
 ['EP061998', '19980730'],
 ['EP061998', '19980730'],
 ['EP061998', '19980730'],
 ['EP061998', '19980730'],
 ['EP061998', '19980731'],
 ['EP061998', '19980731'],
 ['EP061998', '19980731'],
 ['EP061998', '19980731'],
 ['EP061998', '19980801'],
 ['EP061998', '19980801'],
 ['EP061998', '19980801'],
 ['EP061998', '19980801'],
 ['EP061998', '19980802'],
 ['EP061998', '19980802'],
 ['EP061998', '19980802'],
 ['EP061998', '19980802'],
 ['EP061998', '19980803'],
 ['EP061998', '19980803'],
 ['EP061998', '19980803'],
 ['EP061998', '19980803'],
 ['EP061998', '19980804'],
 ['EP061998', '19980804'],
 

 ['EP082005', '20050820'],
 ['EP082005', '20050820'],
 ['EP082005', '20050820'],
 ['EP082005', '20050821'],
 ['EP082005', '20050821'],
 ['EP082005', '20050821'],
 ['EP082005', '20050821'],
 ['EP082005', '20050822'],
 ['EP082005', '20050822'],
 ['EP082005', '20050822'],
 ['EP082005', '20050822'],
 ['EP082005', '20050823'],
 ['EP082005', '20050823'],
 ['EP082005', '20050823'],
 ['EP082005', '20050823'],
 ['EP082005', '20050824'],
 ['EP082005', '20050824'],
 ['EP082005', '20050824'],
 ['EP082005', '20050824'],
 ['EP082005', '20050825'],
 ['EP082005', '20050825'],
 ['EP082005', '20050825'],
 ['EP082005', '20050825'],
 ['EP082005', '20050826'],
 ['EP082005', '20050826'],
 ['EP082005', '20050826'],
 ['EP082005', '20050826'],
 ['EP082005', '20050827'],
 ['EP082005', '20050827'],
 ['EP082005', '20050827'],
 ['EP082005', '20050827'],
 ['EP082005', '20050828'],
 ['EP092005', '20050825'],
 ['EP092005', '20050825'],
 ['EP092005', '20050826'],
 ['EP092005', '20050826'],
 ['EP092005', '20050826'],
 

 ['CP012006', '20060825'],
 ['CP012006', '20060825'],
 ['CP012006', '20060826'],
 ['CP012006', '20060826'],
 ['CP012006', '20060826'],
 ['CP012006', '20060826'],
 ['CP012006', '20060827'],
 ['CP012006', '20060827'],
 ['CP012006', '20060827'],
 ['CP012006', '20060827'],
 ['CP012006', '20060828'],
 ['CP012006', '20060828'],
 ['CP012006', '20060828'],
 ['CP012006', '20060828'],
 ['CP012006', '20060829'],
 ['CP012006', '20060829'],
 ['CP012006', '20060829'],
 ['CP012006', '20060829'],
 ['CP012006', '20060830'],
 ['CP012006', '20060830'],
 ['CP012006', '20060830'],
 ['CP012006', '20060830'],
 ['CP012006', '20060831'],
 ['CP012006', '20060831'],
 ['CP012006', '20060831'],
 ['CP012006', '20060831'],
 ['CP012006', '20060901'],
 ['CP012006', '20060901'],
 ['CP012006', '20060901'],
 ['CP012006', '20060901'],
 ['CP012006', '20060902'],
 ['CP012006', '20060902'],
 ['CP012006', '20060902'],
 ['CP012006', '20060902'],
 ['CP012006', '20060903'],
 ['CP012006', '20060903'],
 ['CP012006', '20060903'],
 

 ['EP062012', '20120711'],
 ['EP062012', '20120711'],
 ['EP062012', '20120711'],
 ['EP062012', '20120711'],
 ['EP062012', '20120712'],
 ['EP062012', '20120712'],
 ['EP062012', '20120712'],
 ['EP062012', '20120712'],
 ['EP062012', '20120713'],
 ['EP062012', '20120713'],
 ['EP062012', '20120713'],
 ['EP062012', '20120713'],
 ['EP062012', '20120714'],
 ['EP062012', '20120714'],
 ['EP062012', '20120714'],
 ['EP062012', '20120714'],
 ['EP062012', '20120715'],
 ['EP062012', '20120715'],
 ['EP062012', '20120715'],
 ['EP062012', '20120715'],
 ['EP062012', '20120716'],
 ['EP062012', '20120716'],
 ['EP062012', '20120716'],
 ['EP062012', '20120716'],
 ['EP062012', '20120717'],
 ['EP062012', '20120717'],
 ['EP062012', '20120717'],
 ['EP062012', '20120717'],
 ['EP062012', '20120718'],
 ['EP062012', '20120718'],
 ['EP062012', '20120718'],
 ['EP062012', '20120718'],
 ['EP062012', '20120719'],
 ['EP062012', '20120719'],
 ['EP062012', '20120719'],
 ['EP062012', '20120719'],
 ['EP062012', '20120720'],
 

 ['EP052013', '20130706'],
 ['EP052013', '20130706'],
 ['EP052013', '20130706'],
 ['EP052013', '20130707'],
 ['EP052013', '20130707'],
 ['EP052013', '20130707'],
 ['EP052013', '20130707'],
 ['EP052013', '20130708'],
 ['EP052013', '20130708'],
 ['EP052013', '20130708'],
 ['EP052013', '20130708'],
 ['EP052013', '20130709'],
 ['EP052013', '20130709'],
 ['EP052013', '20130709'],
 ['EP052013', '20130709'],
 ['EP062013', '20130725'],
 ['EP062013', '20130725'],
 ['EP062013', '20130725'],
 ['EP062013', '20130725'],
 ['EP062013', '20130726'],
 ['EP062013', '20130726'],
 ['EP062013', '20130726'],
 ['EP062013', '20130726'],
 ['EP062013', '20130727'],
 ['EP062013', '20130727'],
 ['EP062013', '20130727'],
 ['EP062013', '20130727'],
 ['EP062013', '20130728'],
 ['EP062013', '20130728'],
 ['EP062013', '20130728'],
 ['EP062013', '20130728'],
 ['EP062013', '20130729'],
 ['EP062013', '20130729'],
 ['EP062013', '20130729'],
 ['EP062013', '20130729'],
 ['EP062013', '20130730'],
 ['EP062013', '20130730'],
 